In [11]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [12]:
from secret import Dagshub_PASSWORD

In [13]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="GyanPrakashKushwaha"
os.environ["MLFLOW_TRACKING_PASSWORD"]= Dagshub_PASSWORD

## Entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

In [15]:
@dataclass(frozen=True)
class MLFlowTrackingConfig:
    mflow_dir : Path
    test_data: Path
    model_obj : str
    metrics_file: str
    params : dict
    mlflow_uri : str
    target_col : str

## configuration 

In [16]:
from churnPredictor.utils import *
from churnPredictor.constants import *

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])

    def get_mlflow_tracking_config(self) -> MLFlowTrackingConfig:
        config = self.config.mlflow_tracking
        params = self.params.RandomForest
        schema = self.schema.DEPENDET_FEATURES

        # create_dirs([self.config.mflow_dir])

        return MLFlowTrackingConfig(
            mflow_dir=config.mlflow_dir,
            test_data=config.test_data,
            model_obj=config.model_obj_path,
            metrics_file=config.metrics_file_name,
            params=params,
            target_col=schema.Churn,
            mlflow_uri='https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow'
        )

        

## component

In [18]:
from churnPredictor.components.model_trainer import ModelTrainer
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, classification_report
import pandas as pd
import joblib
import mlflow
from urllib.parse import urlparse

In [19]:
class TrackModelPerformance:
    def __init__(self,config:MLFlowTrackingConfig):
        self.config = config

    def evaluate(self,true,pred):
        
        cm = confusion_matrix(true, pred)
        accuracy = accuracy_score(true, pred)
        recall = recall_score(true, pred)
        precision = precision_score(true, pred)
        report = classification_report(true, pred)

        evaluation_report = {'confusion_matrix': cm,
                    'accuracy': accuracy,
                    'recall': recall,
                    'precision': precision,
                    'classification_report': report}
        
        logger.info(f'evaluation_report -> {evaluation_report}')
        return evaluation_report
    
    def start_mlflow(self):
        test_data = pd.read_csv(self.config.test_data)
        model = joblib.load(self.config.model_obj)
        logger.info(f'{model} loaded')
        X_test = test_data.drop('Churn',axis=1)
        y_test = test_data['Churn']

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            y_pred = model.predict(X_test)

        evaluation_report = self.evaluate(true=y_test,pred=y_pred)
        # save_json(path=Path(self.config.metrics_file),data=evaluation_report)

        mlflow.log_params(self.config.params)
        mlflow.log_metrics(evaluation_report)

        if tracking_url_type_store != 'file':
            mlflow.sklearn.log_model(model, "model", registered_model_name="random forest")
        else:
            mlflow.sklearn.log_model(model, "model")
    

In [20]:
try:
    config = ConfigurationManager()
    mlflow_tracking_config = config.get_mlflow_tracking_config()
    track_model = TrackModelPerformance(config=mlflow_tracking_config)
    track_model.start_mlflow()
except Exception as e:
    raise CustomException(e)

[2023-09-15 21:32:52,851: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-15 21:32:52,854: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-15 21:32:52,856: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-15 21:32:52,857: INFO: utils: Created artifacts]
[2023-09-15 21:32:53,893: INFO: 3631996127: RandomForestClassifier(n_estimators=132, oob_score=True) loaded]


CustomException: Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_3248\929068938.py] line number [5] error message [Run with UUID 191b8879314744108d766db1a125f25a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True]